## train

In [1]:
import grid2op
from grid2op.Reward import L2RPNReward

# from l2rpn_baselines.utils import TrainingParam
# from l2rpn_baselines.DuelQLeapNet import train 
# from l2rpn_baselines.DuelQLeapNet.LeapNet_NNParam import LeapNet_NNParam
from l2rpn_baselines.utils import TrainingParam
from mnet import train 
from mnet.LeapNet_NNParam import LeapNet_NNParam

import grid2op
from lightsim2grid.LightSimBackend import LightSimBackend

import pathlib

from grid2op.Reward import L2RPNSandBoxScore, L2RPNReward
from l2rpn_baselines.DuelQLeapNet import evaluate

%load_ext autoreload
%autoreload 2

In [2]:
#import inspect
#lines = inspect.getsource(DeepQAgent)
#print(lines)

In [3]:
# define the environment
# env = grid2op.make("l2rpn_case14_sandbox",
#                    reward_class=L2RPNReward)
backend = LightSimBackend()
env = grid2op.make("l2rpn_neurips_2020_track1_large", backend=backend,
                   reward_class=L2RPNReward)

In [4]:
pathic = pathlib.Path().absolute()

In [5]:
# use the default training parameters
tp = TrainingParam()

# this will be the list of what part of the observation I want to keep
# more information on https://grid2op.readthedocs.io/en/latest/observation.html#main-observation-attributes
li_attr_obs_X = ["day_of_week", "hour_of_day", "minute_of_hour", "prod_p", "prod_v", "load_p", "load_q",
                 "actual_dispatch", "target_dispatch", "topo_vect", "time_before_cooldown_line",
                 "time_before_cooldown_sub", "rho", "timestep_overflow", "line_status"]

# neural network architecture
li_attr_obs_X = ["day_of_week", "hour_of_day", "minute_of_hour", "prod_p", "prod_v", "load_p", "load_q",
                 "actual_dispatch", "target_dispatch", "topo_vect", "time_before_cooldown_line",
                 "time_before_cooldown_sub", "timestep_overflow", "line_status", "rho"]
# compared to the other baseline, we have different inputs at different place, this is how we split it
li_attr_obs_Tau = ["rho", "line_status"]
#sizes = [800, 800, 800, 494, 494, 494]
sizes = [1800, 1800, 1800, 994, 994, 994]


# nn architecture
x_dim = LeapNet_NNParam.get_obs_size(env, li_attr_obs_X)
tau_dims = [LeapNet_NNParam.get_obs_size(env, [el]) for el in li_attr_obs_Tau]

kwargs_archi = {'sizes': sizes,
                'activs': ["relu" for _ in sizes],
                'x_dim': x_dim,
                'tau_dims': tau_dims,
                'tau_adds': [0.0 for _ in range(len(tau_dims))],  # add some value to taus
                'tau_mults': [1.0 for _ in range(len(tau_dims))],  # divide by some value for tau (after adding)
                "list_attr_obs": li_attr_obs_X,
                "list_attr_obs_tau": li_attr_obs_Tau
                }

# select some part of the action
# more information at https://grid2op.readthedocs.io/en/latest/converter.html#grid2op.Converter.IdToAct.init_converter
kwargs_converters = {"all_actions": None,
                     "set_line_status": False,
                     "change_bus_vect": True,
                     "set_topo_vect": False
                     }
# define the name of the model
nm_ = "FirstDQLN"
save_path = f'{pathic}'
logs_dir = f'{pathic}/LOGS'

* посмотреть на данные попоробовать отскейлить. Т.к. ошибка слишком большая.
* Уменьшить LR
* Добавить механизм внимания
* 

In [ ]:
try:
    train(env,
          name=nm_,
          iterations=10000000, #bad realisation
          save_path=save_path,
          load_path=save_path,  #If you want to reload your baseline, specify the path where it is located. **NB** if a baseline is reloaded
                                # some of the argument provided to this function will not be used.
          logs_dir=logs_dir,
          training_param=tp,
          kwargs_converters=kwargs_converters,
          kwargs_archi=kwargs_archi);
except Exception as e:
    print(f'{e}!!!!!!')
finally:
    env.close()

INFO: Reloading a model, the architecture parameters will be ignored
INFO: Reloading a model, training parameters will be ignored
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [ ]:
import shutil
shutil.rmtree(f'{pathic}/LOGS')
shutil.rmtree(f'{pathic}/wandb')

## evaluete

In [ ]:
backend = LightSimBackend()
env = grid2op.make("l2rpn_neurips_2020_track1_large", backend=backend,
                   reward_class=L2RPNReward)


# Call evaluation interface
evaluate(env,
         name=nm_,
         load_path=save_path,
         logs_path=logs_dir,
         nb_episode=100,
         nb_process=1,
         max_steps=-1,
         verbose=False,
         save_gif=False)